# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    'study19' : "3ackEOD2vox5Oc1vjeALKJ", 
    'study20' : "3Cmpe4nGzOQDefowW49pBS",
    'PianoJz' : "2IqcxvGty13ewL583z9Kdn",
    'cringe1' : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study20'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100



HTTP Error for PUT to https://api.spotify.com/v1/playlists/3Cmpe4nGzOQDefowW49pBS/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 400-407 --> 191:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/3Cmpe4nGzOQDefowW49pBS/tracks:
 Tracks selected to be reordered are out of bounds, reason: None


HTTP Error for PUT to https://api.spotify.com/v1/playlists/3Cmpe4nGzOQDefowW49pBS/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 2, 0-100 --> 218:
	{'snapshot_id': 'AAABullef5Qdk2oXZX9uyXgzfSR4c2Yq'}
ERROR, 400-407 --> 70:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/3Cmpe4nGzOQDefowW49pBS/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 4, 200-300 --> 311:
	{'snapshot_id': 'AAABux7sqyTA6jjLPZXNKhA0b01chBsu'}

Phase 2: Chunks of 10

Iteration 1, 110-120 --> 67:
	{'snapshot_id': 'AAABvMbKdlKdLeJCRW5Lg7TNyOmmK1k3'}
Iteration 2, 310-320 --> 362:
	{'snapshot_id': 'AAABvevigby2NRcetPU+abwIt7+Y3tvF'}
Iteration 3, 350-360 --> 119:
	{'snapshot_id': 'AAABvhZ2bUKPtDkCpMWGLP6y1wCbCTsf'}
Iteration 4, 260-270 --> 239:
	{'snapshot_id': 'AAABv88D8OEPQMF+AA6mOMZEJj7mVgMn'}
Iteration 5, 0-10 --> 225:
	{'snapshot_id': 'AAABwPGEPuDhnllmxuYKwiF1GTKM2M0A'}
Iteration 6, 360-370 --> 104:
	{'snapshot_id': 'AAABwS9oQiah0KwA6P64J2KV8CerPzxH'}
Iteration 7, 360-370 --> 235:
	{'snapshot_id': 'AAABwmxrTASP6NbSodKnz7A/wh2O/lr0'}
Iteration 8, 120-130 --> 372:
	{'snapshot_id': 'A

HTTP Error for PUT to https://api.spotify.com/v1/playlists/3Cmpe4nGzOQDefowW49pBS/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 400-407 --> 326:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/3Cmpe4nGzOQDefowW49pBS/tracks:
 Tracks selected to be reordered are out of bounds, reason: None

Phase 3: Chunks of 5, 1/2

Iteration 1, 45-50 --> 150:
	{'snapshot_id': 'AAAB4olWF1gFAZGRn4MJhqe54bnVyAnv'}
Iteration 2, 75-80 --> 293:
	{'snapshot_id': 'AAAB4woAhIvkuxbOt4YogHhC9CAO9kfr'}
Iteration 3, 335-340 --> 26:
	{'snapshot_id': 'AAAB5I+wiz8OkDqQDBvSwrx7n3w/jOZp'}
Iteration 4, 145-150 --> 255:
	{'snapshot_id': 'AAAB5bZvrugX1+D5zNzK6Zfvp5st9P4Q'}
Iteration 5, 385-390 --> 142:
	{'snapshot_id': 'AAAB5hkNSXwbs8DML8mCIE0sJzfRxlDS'}
Iteration 6, 310-315 --> 280:
	{'snapshot_id': 'AAAB5966KglcN3QOxLaWx8Lx5qM+KCMU'}
Iteration 7, 275-280 --> 64:
	{'snapshot_id': 'AAAB6FJe4I9RGYHdfeL/TQdfDSmzIUSc'}
Iteration 8, 115-120 --> 38:
	{'snapshot_id': 'AAAB6aV+UpnzNaTqpRLxBEK/XlH8i6MJ'}
Iteration 9, 40-45 --> 11:
	{'snapshot_id': 'AAAB6knUeBS71Y60cIWqK6Z/gOHd/+28'}
Iteration 10, 10-15 --> 82:
	{'snapshot_id': 'AA